In [211]:
from skimage.io import imread_collection , concatenate_images
from sklearn.neighbors import NearestNeighbors
from sklearn.neighbors import KNeighborsClassifier
from sklearn import preprocessing
from sklearn.utils import shuffle 
import random
import numpy as np
import matplotlib.pyplot as plt
import pywt as ptw

import copy as cp
from skimage.io.manage_plugins import _inject_imread_collection_if_needed




In [212]:
def draw(image):
    plt.imshow(image)
    plt.show()
    return

def hwavelet(images, n):
    cA = images
    for i in range(n):
        cA, (cH, cV, cD) = ptw.dwt2(cA, 'haar')
    return cA

def normalizar(dataset):
    aux = []
    for i in range(len(dataset)):
        aux = aux + [dataset[i].reshape(24*24).tolist()]
    return aux

def load(emotion, emotion_arr):
    aux = imread_collection(emotion_arr)
    aux_lb = [emotion] * len(aux)

    for i in range(len(aux)):
        all_images.append(aux[i])
        all_labels.append(aux_lb[i])



In [213]:
anger = 'images/anger/*.png'

contempt = 'images/contempt/*.png'

disgust = 'images/disgust/*.png'

fear = 'images/fear/*.png'

happy = 'images/happy/*.png'

sadness = 'images/sadness/*.png'

surprise = 'images/surprise/*.png'


all_images = []
all_labels = []

load('anger', anger)
load('contempt', contempt)
load('disgust', disgust)
load('fear', fear)
load('happy', happy)
load('sadenss', sadness)
load('surprise', surprise)

all_images_processed = hwavelet(all_images,1)

all_images_processed_normalized = normalizar(all_images_processed)

print(len(all_images))
print(len(all_labels))


981
981


In [214]:
def bootstrap(k,n):
    tam = len(all_images_processed) 
    t = tam
    ps = []
    for i in range(tam):
        ps.append((all_images_processed[i], all_images_processed_normalized[i], all_labels[i]))
    ds = shuffle(ps)
    total_aciertos = 0
    total_samples = n
    if n==0:
        total_samples = 0
    
    for i in range(k):
        if n==0:
            n = random.randint(0,tam)
            total_samples += n
        idxs = random.sample(range(0,tam), n)
        
        train_i = []
        train_l = []
        test_i = []
        test_l = []

        for i in range(tam):
            if i in idxs:
                test_i.append(ds[i][1])
                test_l.append(ds[i][2])
            else:
                train_i.append(ds[i][1])
                train_l.append(ds[i][2])


        knn = KNeighborsClassifier(n_neighbors=1)

        knn.fit(train_i, train_l)

        aciertos = 0
        resultados = []
        for i in range(len(test_i)):
            predict = knn.predict([test_i[i]])
            if predict == test_l[i]:
                aciertos += 1
            #draw(test_im[i])
            #resultados.append('predict: '+ str(predict)+ ', real: ',str(test_l[i]))
            #if(i%3==0): print('predict: '+ str(predict)+ ', real: ',str(test_l[i]))
        total_aciertos += aciertos
    
    total_aciertos = total_aciertos/k

    print('Aciertos totales:', round(total_aciertos,2),'de:', round(total_samples,2), 'Accuracy: ' + str(round( ((total_aciertos/total_samples)*100), 2)) + '%')      

def kfold(k):
    tam = len(all_images_processed) 
    t = tam
    '''
    if tam % k != 0:
        print('choose another n')
        return
    '''
    diff = int(t/k)

    ps = []

    for i in range(tam):
        ps.append((all_images_processed[i], all_images_processed_normalized[i], all_labels[i]))

    ds = shuffle(ps)

    #print(len(ds[1]))
    
    a = 0
    total_aciertos = 0

    
    for i in range(k):

        train_i = []
        train_l = []
        
        test_i = []
        test_l = []
        
        for i in range(t):
            
            if i in range(a,a+diff):
                test_i.append(ds[i][1])
                test_l.append(ds[i][2])
                
            else:
                train_i.append(ds[i][1])
                train_l.append(ds[i][2])
                
                
        a += diff

        knn = KNeighborsClassifier(n_neighbors=1)

        knn.fit(train_i, train_l)

        aciertos = 0
        resultados = []
        for i in range(len(test_i)):
            predict = knn.predict([test_i[i]])
            if predict == test_l[i]:
                aciertos += 1
            #draw(test_im[i])
            #resultados.append('predict: '+ str(predict)+ ', real: ',str(test_l[i]))
            #if(i%3==0): print('predict: '+ str(predict)+ ', real: ',str(test_l[i]))
               

        total_aciertos += aciertos
    
    total_aciertos = total_aciertos/k

    print('Aciertos totales:', round(total_aciertos,2),'de:', round(diff,3), 'Accuracy: ' + str(round( ((total_aciertos/diff)*100), 2)) + '%')

    
    


    


In [215]:
kfold_folds = 3
kfold(kfold_folds)

bootstrap_folds = 6
test_size = 300 #if 0, test_size randomizes every fold, train = ds-test_size
bootstrap(bootstrap_folds,test_size)

Aciertos totales: 294.67 de: 327 Accuracy: 90.11%
Aciertos totales: 281.83 de: 300 Accuracy: 93.94%
